<a href="https://colab.research.google.com/github/Vlasovets/Deep_learning_course_assistantship/blob/master/layout_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 1.x # Oleg's code

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.x # Oleg's code`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [4]:
import os
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

# Some GPUs require setting the `allow_growth` setting.
# Comment out this code is you don't have a GPU card.
import tensorflow.compat.v1 as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config=config)

In [0]:
!unzip layouts-easy.zip #Oleg unzips the folder

In [19]:
dataset_dir = 'layouts-easy'
trainer_dir = dataset_dir + '/train'

# Specify manually the class labels, otherwise Keras will assign alphanumeric values
# as directories are listed (in no particular order).
dirname, class_labels, _ = next(os.walk(trainer_dir))
print('Class labels: {}'.format(class_labels))

# Don't compile the model when testing new data.
# model = load_model('{}.h5'.format(dataset_dir), compile=False)

Class labels: ['Good', 'Bad']


In [0]:
model = load_model('{}.h5'.format(dataset_dir), compile=False) #Oleg compile the trained model

In [26]:
# Don't shuffle the data, so that we can get the groundtruth labels later.
test_generator = ImageDataGenerator().flow_from_directory(
    dataset_dir + '/test',
    shuffle=False,
    color_mode='grayscale',
    target_size=(200, 200),
    class_mode='binary',
    classes=class_labels
)

Found 100 images belonging to 2 classes.


In [0]:
# Since we have groundtruth labels, we can use `model.evaluate_generator` here.
# However in production we don't have groundtruth so let's use the model to predict new data
# and then compare the predictions against the test labels.
y_true = list(map(lambda f : os.path.dirname(f), test_generator.filenames))
probas = model.predict_generator(test_generator)
y_pred = list(map(lambda p : class_labels[0] if p < 0.5 else class_labels[1], probas))

In [28]:
# Let's see how good those predictions were.
precision, recall, fmeasure, _ = precision_recall_fscore_support(y_true, y_pred, labels=class_labels, average='weighted')
print('Precision: {:.2f}%'.format(precision * 100))
print('Recall: {:.2f}%'.format(recall * 100))
print('F-measure: {:.2f}%'.format(fmeasure * 100))

# Finally compute the ROC AUC to see the discriminative power of the model.
# We'll use the `roc_auc_score()` function, which expects *binary* groundtruth labels and probability estimates of the *positive class*;
# so be careful how labels and probabilities are defined.
binary_labels = [p == y_true[i] for i, p in enumerate(y_pred)]
auc = roc_auc_score(binary_labels, 1 - probas, average='weighted')
print('AUC: {:.2f}%'.format(auc * 100))

Precision: 77.10%
Recall: 66.00%
F-measure: 62.12%
AUC: 60.56%
